A script for making automated scanning jobs for the HP Envy 4500 printer.

Reverse-engineered the webscan service att http://192.168.1.17/#hId-pgWebScan

In [47]:
import requests
import shutil
from PIL import Image
import xml.etree.ElementTree as ET


In [48]:
# printer ip address
PRINTER = "http://192.168.1.17"
SCHEMA = "{http://www.hp.com/schemas/imaging/con/ledm/jobs/2009/04/30}"

cookies = {
    "sid": "s0538708d-811ba985e173db93169e975baf2840b7",
    "mobileView": "0",
}
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:101.0) Gecko/20100101 Firefox/101.0",
    "Accept": "application/xml, text/xml, */*",
    "Accept-Language": "en,sv-SE;q=0.8,sv;q=0.5,en-US;q=0.3",
    # 'Accept-Encoding': 'gzip, deflate',
    "DNT": "1",
    "Connection": "keep-alive",
    "Referer": PRINTER,
    # Requests sorts cookies= alphabetically
    # 'Cookie': 'sid=s0538708d-811ba985e173db93169e975baf2840b7; mobileView=0',
    "Pragma": "no-cache",
    "Cache-Control": "no-cache",
}



# Get scanner status

In [49]:
response = requests.get(
    f"{PRINTER}/Scan/Status", headers=headers
)

root = ET.fromstring(response.content)
for child in root.iter():
    if child.tag.endswith("ScannerState"):
        break
else:
    print("Could not get ScannerState")

if child.text != "Idle":
    print(f"Scanner is not idle: {child.text}")


# Start scanning job

In [50]:
dpi = 600 # higher is better, in range 75-600
compression = 0 # lower is better - 0 = no compression, 100 = full compression

scan_post = f"""
<scan:ScanJob xmlns:scan="http://www.hp.com/schemas/imaging/con/cnx/scan/2008/08/19" xmlns:dd="http://www.hp.com/schemas/imaging/con/dictionaries/1.0/" xmlns:fw="http://www.hp.com/schemas/imaging/con/firewall/2011/01/05">
    <scan:XResolution>{dpi}</scan:XResolution>
    <scan:YResolution>{dpi}</scan:YResolution>
    <scan:XStart>0</scan:XStart>
    <scan:YStart>0</scan:YStart>
    <scan:Width>2480</scan:Width>
    <scan:Height>3508</scan:Height>
    <scan:Format>Jpeg</scan:Format>
    <scan:CompressionQFactor>{compression}</scan:CompressionQFactor>
    <scan:ColorSpace>Color</scan:ColorSpace>
    <scan:BitDepth>8</scan:BitDepth>
    <scan:InputSource>Platen</scan:InputSource>
    <scan:GrayRendering>NTSC</scan:GrayRendering>
    <scan:ToneMap>
        <scan:Gamma>1000</scan:Gamma>
        <scan:Brightness>1000</scan:Brightness>
        <scan:Contrast>1000</scan:Contrast>
        <scan:Highlite>179</scan:Highlite>
        <scan:Shadow>25</scan:Shadow>
    </scan:ToneMap>
    <scan:ContentType>Photo</scan:ContentType>
</scan:ScanJob>
""".strip()

response = requests.post(
    f"{PRINTER}/Scan/Jobs",
    cookies=cookies,
    headers={"Content-Type": "text/xml", **headers},
    data=scan_post,
)
response.raise_for_status()
response


<Response [201]>

# Get job list

In [51]:
response = requests.get(f"{PRINTER}/Jobs/JobList", cookies=cookies, headers=headers)
response.raise_for_status()
root = ET.fromstring(response.content)
jobs = [{attr.tag.removeprefix(SCHEMA): attr.text for attr in job} for job in root]
scan_jobs = [job for job in jobs if job["JobCategory"] == "Scan"]
scan_jobs

[
    {
        'JobUrl': '/Jobs/JobList/8',
        'JobCategory': 'Scan',
        'JobState': 'Canceled',
        'JobStateUpdate': '1456-26',
        'JobSource': 'userIO'
    },
    {
        'JobUrl': '/Jobs/JobList/9',
        'JobCategory': 'Scan',
        'JobState': 'Completed',
        'JobStateUpdate': '1456-30',
        'JobSource': 'userIO'
    },
    {
        'JobUrl': '/Jobs/JobList/10',
        'JobCategory': 'Scan',
        'JobState': 'Completed',
        'JobStateUpdate': '1456-34',
        'JobSource': 'userIO'
    },
    {
        'JobUrl': '/Jobs/JobList/11',
        'JobCategory': 'Scan',
        'JobState': 'Processing',
        'JobStateUpdate': '1456-37',
        'JobSource': 'userIO',
        '{http://www.hp.com/schemas/imaging/con/cnx/scan/2008/08/19}ScanJob': '\n   '
    }
]

# Get job status

In [52]:
job = scan_jobs[-1]
response = requests.get(
    f'{PRINTER}{job["JobUrl"]}', cookies=cookies, headers=headers
)
response.raise_for_status()
root = ET.fromstring(response.content)
job = {attr.tag.removeprefix(SCHEMA): attr.text for attr in root}
print(job)
response
# "Processing"

{'JobUrl': '/Jobs/JobList/11', 'JobCategory': 'Scan', 'JobState': 'Processing', 'JobStateUpdate': '1456-37', 'JobSource': 'userIO', '{http://www.hp.com/schemas/imaging/con/cnx/scan/2008/08/19}ScanJob': '\n   '}


<Response [200]>

# Download scanned page

In [53]:
filename = "scan.jpg"
job_id = job["JobUrl"].split("/")[-1]

with requests.get(
    f"{PRINTER}/Scan/Jobs/{job_id}/Pages/1",
    cookies=cookies,
    headers=headers,
    stream=True,
) as r:
    with open(filename, "wb") as f:
        shutil.copyfileobj(r.raw, f)


# Resize image

In [ ]:
def downscale(filename: str, size: int, quality: int):
    with Image.open(filename) as im:
        im.thumbnail((size, size))
        [file, ext] = filename.rsplit(".", 1)
        im.save(f"{file}_{size}p_{quality}q.{ext}", quality=quality)

downscale(filename, 2560, 65)